# Laboratorium 4 - Singular Value Decomposition

##### Aleksandra Mazur

## Zadanie 1 Wyszukiwarka

#### 1. 
Przygotuj duży (> 1000 elementów) zbiór dokumentów tekstowych w języku angielskim
(np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów
HTML uzyskanych za pomoca Web crawlera, zbiór rozdziałów wyciętych z
różnych książek).

Ze strony https://ebible.org/find/details.php?id=eng-web&all=1 pobrano zbiór dokumentów tekstowych w języku angielskim, liczący 1100 plików.

Poniżej przedstawiono kilka przykładowych plików.

In [1]:
import numpy as np
import os
import io

files = os.listdir('documents/')

In [2]:
def show_two_files(files):
    i = 0
    for file in files:
        f = io.open('documents/' + file, encoding="utf8")
        text_from_file = f.read()
        print("File number: ", i)
        print(text_from_file)
        i += 1
        if i == 2:
            break

In [3]:
show_two_files(files)

File number:  0
﻿This set of files contains a script of canonical text, chapter by chapter,
for the purpose of reading to make an audio recording.
All footnotes, introductions, and verse numbers have been stripped out.


File number:  1
﻿The First Book of Moses, Commonly Called Genesis.
Chapter 1.
In the beginning, God created the heavens and the earth. 
The earth was formless and empty. Darkness was on the surface of the deep and God’s Spirit was hovering over the surface of the waters. 
God said, “Let there be light,” and there was light. 
God saw the light, and saw that it was good. God divided the light from the darkness. 
God called the light “day”, and the darkness he called “night”. There was evening and there was morning, the first day. 
God said, “Let there be an expanse in the middle of the waters, and let it divide the waters from the waters.” 
God made the expanse, and divided the waters which were under the expanse from the waters which were above the expanse; and it was s

#### 2. 
Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów
cech bag-of-words (indeksacja). Przykładowo zbiorem takim może być unia wszystkich
słów występujących we wszystkich tekstach.

In [4]:
import re
import nltk
nltk.download('punkt')

Słownik słów kluczowych określono jako unię wszystkich słów występujących we wszystkich tekstach.

In [5]:
def create_dictionary(files):
    words_with_quantity = {}
    dictionary = []
    
    for file in files:
        f = io.open('documents/' + file, encoding="utf8")
        text_from_file = f.read()
        f.close()
        sentences = text_from_file.split('\n')
        
        for sentence in sentences:
            sentence = sentence.lower()
            sentence = re.sub(r'[^\w\s]', '', sentence)
            sentence = re.sub('[0-9]', '', sentence)
            words = nltk.word_tokenize(sentence)
            dictionary.extend(words)
            
            for word in words:
                if word in words_with_quantity.keys():
                    words_with_quantity[word] += 1
                else:
                    words_with_quantity[word] = 1
                    
    dictionary = sorted(list(set(dictionary)))
    return dictionary, words_with_quantity

In [6]:
dictionary, words_with_quantity = create_dictionary(files)

Poniżej przedstawiono 10 słów, które najczęściej pojawiały się w plikach tekstowych wraz z ilością ich występowania.

In [8]:
def show_most_popular_words(quantity, words_with_quantity):
    words_list = sorted(words_with_quantity.items(), key = lambda x: x[1], reverse=True)
    print("Liczba słów w słowniku: ", len(words_with_quantity))
    print(quantity, " najpopularniejszych słów:")
    for i, element in enumerate(words_list):
        print(element[0], " -> ", element[1])
        if i == quantity - 1:
            break

In [9]:
show_most_popular_words(10, words_with_quantity)

Liczba słów w słowniku:  13675
10  najpopularniejszych słów:
the  ->  53774
of  ->  31534
and  ->  30339
to  ->  19633
you  ->  12157
in  ->  12045
he  ->  9289
will  ->  9088
a  ->  8511
for  ->  8391


#### 3.
Dla każdego dokumentu j wyznacz wektor cech bag-of-words dj zawierający częstości
występowania poszczególnych słów (termów) w tekście.

In [10]:
def create_vector(file, dictionary):
    bow = {}
    for word in dictionary:
        bow[word] = 0
    f = io.open('documents/' + file, encoding="utf8")
    text_from_file = f.read()
    f.close()
    sentences = text_from_file.split('\n')

    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub(r'[^\w\s]', '', sentence)
        sentence = re.sub('[0-9]', '', sentence)
        words = nltk.word_tokenize(sentence)
        dictionary.extend(words)

        for word in words:
            bow[word] += 1
    return bow

In [11]:
def bag_of_words(quantity, files, dictionary):
    for i, file in enumerate(files):
        print ("\nFile number: ", i)
        bow = create_vector(file, dictionary)
        bow_sorted = sorted(bow.items(), key = lambda x: x[1], reverse=True)
        for element in bow_sorted:
            if (element[1] == 0):
                break
            print(element[0], " -> ", element[1])

        if i == quantity -1:
            break

Poniżej znajdują się wektory cech bag-of-words dla dwóch pierwszych plików. Przy wyświetlaniu pominięto słowa, które nie występowały w danym tekście.

In [12]:
bag_of_words(2, files, dictionary)


File number:  0
of  ->  3
chapter  ->  2
a  ->  1
all  ->  1
an  ->  1
and  ->  1
audio  ->  1
been  ->  1
by  ->  1
canonical  ->  1
contains  ->  1
files  ->  1
footnotes  ->  1
for  ->  1
have  ->  1
introductions  ->  1
make  ->  1
numbers  ->  1
out  ->  1
purpose  ->  1
reading  ->  1
recording  ->  1
script  ->  1
set  ->  1
stripped  ->  1
text  ->  1
the  ->  1
this  ->  1
to  ->  1
verse  ->  1

File number:  1
the  ->  87
and  ->  52
god  ->  30
was  ->  29
earth  ->  20
it  ->  18
of  ->  18
there  ->  17
let  ->  14
to  ->  14
in  ->  13
that  ->  12
after  ->  11
over  ->  11
waters  ->  11
day  ->  10
kind  ->  10
light  ->  10
said  ->  10
their  ->  10
be  ->  9
every  ->  9
expanse  ->  9
on  ->  9
sky  ->  9
saw  ->  8
them  ->  8
good  ->  7
called  ->  6
evening  ->  6
he  ->  6
morning  ->  6
so  ->  6
which  ->  6
a  ->  5
created  ->  5
from  ->  5
made  ->  5
birds  ->  4
darkness  ->  4
fruit  ->  4
have  ->  4
living  ->  4
night  ->  4
with  ->  4
yielding 

#### 4.
Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory
cech ułożone są kolumnowo A m×n = [d1|d2| . . . |dn] (m jest liczbą termów w
słowniku, a n liczbą dokumentów)

In [13]:
def create_matrix(files, dictionary):
    matrix = []
    vector = {}
    for file in files:
        vector = create_vector(file, dictionary)
        matrix.append(vector)
    return matrix

In [14]:
matrix = create_matrix(files, dictionary)

In [15]:
print("Liczba wierszy: ", len(matrix))
print("Liczba kolumn: ", len(matrix[0]))

Liczba wierszy:  1100
Liczba kolumn:  13675


Jak widać liczba wierszy macierzy odpowiada liczbie plików tekstowych, a liczba kolumn zgadza się z ilością słów znajdujących się w zdefiniowanym wcześniej słowniku.

#### 5.
Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez
inverse document frequency. Operacja ta pozwoli na redukcje znaczenia często występujących
słów.

In [ ]:
def count_documents_with_word(word):
    counter = 0
    for doc in range(len(matrix)):
        arr = matrix[doc]
        if arr.get(word) != 0:
            counter += 1
    return counter

In [ ]:
import math

def inverse_document_frequency(words_with_quantity):
    dict = words_with_quantity.keys()
    number_of_documents = len(matrix)
    for word in dict:
        documents_with_word = count_documents_with_word(word)
        if documents_with_word == 0:
            idf = 0
        else:
            idf = float(math.log(number_of_documents/documents_with_word))
        for doc in range(len(matrix)):
            line = matrix[doc]
            line[word] = line.get(word) * idf

In [ ]:
inverse_document_frequency(words_with_quantity)

In [ ]:
vector = matrix[1]
list_words = sorted(vector.items() ,  key=lambda x: x[1], reverse=True)

for elem in list_words :
    if elem[1] == 0:
        break
    print(elem[0] , " -> " , elem[1] )

In [20]:
vector = matrix[1]
list_words = sorted(vector.items() ,  key=lambda x: x[1], reverse=True)

for elem in list_words :
    if elem[1] == 0:
        break
    print(elem[0] , " -> " , elem[1] )

In [21]:
import math

def inverse_document_frequency(words_with_quantity):
    dict = words_with_quantity.keys()
    number_of_documents = len(matrix)
    for word in dict:
        documents_with_word = count_documents_with_word(word)
        if documents_with_word == 0:
            idf = 0
        else:
            idf = float(math.log(number_of_documents/documents_with_word))
        for doc in range(len(matrix)):
            line = matrix[doc]
            line[word] = line.get(word) * idf

In [22]:
inverse_document_frequency(words_with_quantity)

In [19]:
vector = matrix[1]
list_words = sorted(vector.items() ,  key=lambda x: x[1], reverse=True)

for elem in list_words :
    if elem[1] == 0:
        break
    print(elem[0] , " -> " , elem[1] )

expanse  ->  48.54264791717126
kind  ->  32.18875824868201
yielding  ->  23.61781268047341
waters  ->  22.4424291137921
light  ->  20.332521592104612
sky  ->  19.94016344403974
earth  ->  19.495738771115274
seeds  ->  17.71335951035506
creeps  ->  16.180882639057085
lights  ->  16.180882639057085
created  ->  15.982014845080709
evening  ->  15.292308975197725
god  ->  12.756123031962455
creatures  ->  12.338081102670891
was  ->  12.170030862203218
let  ->  11.58077464152981
morning  ->  11.382719909315288
saw  ->  11.326534403089699
birds  ->  11.193491357581983
divide  ->  10.80560423137292
after  ->  10.53866159225495
image  ->  10.343152191891145
darkness  ->  10.148629360527513
moves  ->  10.114310619462298
everything  ->  10.096437897180227
multiply  ->  10.096437897180227
fruit  ->  9.591581091193483
living  ->  9.591581091193483
surface  ->  9.589208907048427
rule  ->  9.273127360074948
seed  ->  9.273127360074948
herb  ->  9.210340371976184
herbs  ->  9.210340371976184
over  ->